# Laboratorium 5: Segmentacja polipów — Architektura typu U-Net(Kvasir-SEG)

W tym ćwiczeniu chcemy nauczyć się budowania i trenowania modeli **segmentacji semantycznej** z wykorzystaniem architektury **U-Net** oraz frameworka **PyTorch Lightning** na zbiorze danych medycznych **Kvasir-SEG** (segmentacja polipów jelitowych w obrazach endoskopowych).

**Główne zagadnienia:**
- Implementacja `LightningDataModule`: podział train/val/test, augmentacje spójne dla obrazu i maski.
- Budowa architektury **U-Net 2D** z enkoderem i dekoderem oraz skip connections.
- Funkcje straty dla segmentacji: **BCEWithLogitsLoss** i **Dice Loss**.
- Metryki: **Dice Score** i **IoU** (Intersection over Union) z biblioteki torchmetrics.
- Wykorzystanie **Transfer Learning** ResNet jako encoder w U-Net.

**Dataset:** 
Użyjemy **Kvasir-SEG** – zbioru zawierającego obrazy endoskopowe jelita grubego wraz z maskami segmentacji polipów. Dataset ten został stworzony do oceny algorytmów segmentacji w endoskopii i zawiera około 1000 par obraz-maska. Dane zostaną **automatycznie pobrane** w notebooku. Skalujemy je również do rozdzielczości 224x224 aby ograniczyć czas potrzebny na uczenie sieci.

## 0) Instalacja i importy

Jeśli chcemy powtarzalnych wyników możemy ustawić stały SEED.

In [1]:
# !pip -q install pytorch-lightning torchmetrics scikit-image opencv-python pillow --extra-index-url https://download.pytorch.org/whl/cpu

import os, zipfile, urllib.request, random, glob, math
from pathlib import Path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import ssl

import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning import LightningModule, LightningDataModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torchmetrics.classification import BinaryJaccardIndex, BinaryF1Score

import torchvision.models as models
from torchvision.models import ResNet18_Weights


# SEED = 42
SEED = None
def set_seed(s=SEED):
    random.seed(s); np.random.seed(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)

if SEED is not None:
    set_seed()
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device, '| Lightning:', pl.__version__)

Device: cuda | Lightning: 2.5.5


## 1) Pobranie i przygotowanie Kvasir-SEG

**Kvasir-SEG** to zbiór danych zawierający obrazy endoskopowe jelita grubego wraz z maskami segmentacji polipów. Poniższy fragment automatycznie pobiera zbiór i sprawdza liczbę pobranych plików. 

Należy zwrócić uwagę, że dla zadania segmentacji zarówno obrazy wejściowe jak i groundtruth są obrazami. W przypadku segmentacji binarnej (tło-obiekt) maska ma wartości binarne. Wymusza to delikatnie inne podejście do zarządzania i przetwarzania wstępnego takich danych.

In [2]:
DATA_ROOT = Path('./data/data_kvasir')
DATA_ROOT.mkdir(parents=True, exist_ok=True)

ZIP_URL = 'https://datasets.simula.no/downloads/kvasir-seg.zip'
ZIP_PATH = DATA_ROOT / 'kvasir.zip'
EXTRACT_DIR = DATA_ROOT / 'Kvasir-SEG'

if not EXTRACT_DIR.exists():
    print('Pobieranie Kvasir-SEG (~150 MB)…')
    ssl._create_default_https_context = ssl._create_unverified_context
    urllib.request.urlretrieve(ZIP_URL, ZIP_PATH)
    print('Rozpakowywanie…')
    with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
        zf.extractall(DATA_ROOT)
    print('Gotowe:', EXTRACT_DIR)
else:
    print('Dane już dostępne:', EXTRACT_DIR)

IMG_DIR = EXTRACT_DIR / 'images'
MSK_DIR = EXTRACT_DIR / 'masks'
imgs = sorted(glob.glob(str(IMG_DIR / '*.jpg')))
msks = sorted(glob.glob(str(MSK_DIR / '*.jpg')))
print('Liczba obrazów/masek:', len(imgs), len(msks))
assert len(imgs) == len(msks) and len(imgs) > 0, 'Brak danych lub brak par obraz–maska.'

Dane już dostępne: data/data_kvasir/Kvasir-SEG
Liczba obrazów/masek: 1000 1000


## Zadanie 1 – Dataset i DataModule dla segmentacji

W tym zadaniu zaimplementujesz własny Dataset oraz DataModule dla danych segmentacyjnych. Kluczową różnicą w porównaniu do klasyfikacji jest to, że augmentacje zmieniające pozycję obiektów (wszystkie przekształcenia geometryczne) muszą być stosowane zarówno do obrazu jak i maski – np. jeśli obracamy obraz o 10 stopni, to maskę również musimy obrócić o dokładnie ten sam kąt.

**Dataset dla segmentacji:**
1. Zaimplementuj klasę `KvasirDataset`, która dziedziczy po `Dataset`.
2. W metodzie `__init__` zapamiętaj ścieżki do obrazów (`img_paths`) i masek (`msk_paths`), oraz opcjonalny obiekt transform.
3. W metodzie `__len__` zwróć liczbę obrazów.
4. W metodzie `__getitem__`:
   - Wybierz odpowiednie ścieżki z zapamiętanych list podczas inicjalizacji.
   - Wczytaj obraz i maskę używając funkcji pomocniczej `load_pair`.
   - Jeśli `self.transform` istnieje, zastosuj go do obu (obraz, maska).
   - Zwróć parę (obraz, maska).

**Transform dla segmentacji:**
1. Zaimplementuj klasę `SegmentationTransform`, która będzie stosować te same transformacje do obrazu i maski. W przypadku segmentacji musimy sami ją zaimplementować, aby móć stosować spójną segmentację dla obu obrazów.
2. W metodzie `__init__`:
   - Zapamiętaj `size` (docelowy rozmiar obrazu).
   - Zapamiętaj `augment` (flaga czy stosować augmentacje, domyślnie False).
   - Stwórz obiekt do konwersji PIL do tensor: `transforms.ToTensor()`.
   - Stwórz **dwa** obiekty do resize: `transforms.Resize(size, ...)`. Pierwszy z interpolacją BILINEAR dla obrazu, a drugi z interpolacją NEAREST dla maski. Jest to ważne, gdyż maska jest binarna i nie chcemy rozmywać jej krawędzi.
   - Stwórz obiekt normalizacji: `transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])` – przesunie wartości z [0,1] do [-1,1].
3. W metodzie `__call__(self, img, mask)`:
   - Wykonaj resize dla obrazu i maski (na obrazach PIL).
   - Skonwertuj oba do tensorów (wartości [0, 1]).
   - Zbinaryzuj maskę: `mask = (mask > 0).float()` – chcemy, żeby maska zawiera tylko 0 i 1, ale miała typ zmiennoprzecinkowy.
   - Jeśli `self.augment == True`:
     - Z prawdopodobieństwem 50% wykonaj horizontal flip (dla obu).
     - Z prawdopodobieństwem 50% wykonaj losową rotację o kąt z przedziału [-10, 10] stopni. Zrób to samo dla obu obrazów, ale z różną interpolacją. Wykorzystaj `transforms.functional.rotate` i podaj argument `interpolation`.
   - Znormalizuj obraz (nie maskę) do [-1, 1].
   - Zwróć `(img.float(), mask.float())`.

**DataModule:**
1. Zaimplementuj klasę `KvasirDataModule`, która dziedziczy po `LightningDataModule`.
2. W metodzie `__init__`:
   - Zapamiętaj listy obrazów i masek.
   - Zapamiętaj `batch`, `nw` (num_workers), `train_split`, `val_split`, `img_size`.
   - Stwórz dwa obiekty transformacji: `self.train_transform` (z augment=True) i `self.val_test_transform` (z augment=False) jako obiekty zaimplementowanej wcześniej klasy.
3. W metodzie `setup(self, stage=None)`:
   - Wylosuj indeksy i podziel dane na train/val/test zgodnie z `train_split` i `val_split`. W tym celu stwórz listę indeksów od 0 do liczby próbek, a następnie wymieszaj je za pomocą `np.random.shuffle`. Podziel wynikowy wektor na 3 częsci zgodnie za przekazanym podziałem na część treningową, walidacyjną i testową. Następnie podziel również odpowiednio ścieżki do plików zgodnie z przygotowanymi listami indeksów.
   - Stwórz i zapamiętaj trzy datasety (treningowy, walidacyjny i testowy) z odpowiednimi transformacjami i zawierające próbki zgodnie w wcześniejszym losowaniem.
   - Wyświetl rozmiary zbiorów.
4. Zaimplementuj metody `train_dataloader()`, `val_dataloader()`, `test_dataloader()`:
   - Zwróć DataLoader z odpowiednim datastem.
   - Ustaw `shuffle=True` tylko dla train.
   - Ustaw `pin_memory=True` jeśli CUDA dostępne.

**Poza klasami:**
1. Stwórz instancję `KvasirDataModule` z listami `imgs` i `msks` (były stworzone we wcześniejszej komórce), rozmiarem batcha i liczbą workerów. Wartości te trzeba dobrać na podstawie specyfikacji maszyny na której uruchamiany będzie trening.
2. Wywołaj `dm.setup()`.
3. Pobierz jeden batch z train loadera (`next(iter(dm.train_dataloader()))`) i wyświetl kształty. Powinny pojawić się kształty typu `(16, 3, 384, 384)` dla obrazów i `(16, 1, 384, 384)` dla masek.

In [3]:
class KvasirDataset(Dataset):
    def __init__(self, img_paths, msk_paths, transform=None):
        self.img_paths = img_paths
        self.msk_paths = msk_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        msk_path = self.msk_paths[idx]
        img = Image.open(img_path).convert('RGB')
        mask = Image.open(msk_path).convert('L')
        if self.transform:
            img, mask = self.transform(img, mask)
        else:
            to_tensor = transforms.ToTensor()
            img = to_tensor(img).float()
            mask = to_tensor(mask).float()
            mask = (mask > 0).float()
        
        return img, mask

class SegmentationTransform:
    def __init__(self, size=(384, 384), augment=False):
        self.size = size
        self.augment = augment
        self.to_tensor = transforms.ToTensor()
        self.img_resize = transforms.Resize(size, interpolation=transforms.InterpolationMode.BILINEAR)
        self.mask_resize = transforms.Resize(size, interpolation=transforms.InterpolationMode.NEAREST)
        self.normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    
    def __call__(self, img, mask):
        img = self.img_resize(img)
        mask = self.mask_resize(mask)
        img = self.to_tensor(img)
        mask = self.to_tensor(mask)
        mask = (mask > 0).float()

        if self.augment:
            if random.random() > 0.5:
                img = transforms.functional.hflip(img)
                mask = transforms.functional.hflip(mask)
            
            if random.random() > 0.5:
                angle = random.uniform(-10, 10)
                img = transforms.functional.rotate(
                    img, angle, interpolation=transforms.InterpolationMode.BILINEAR
                )
                mask = transforms.functional.rotate(
                    mask, angle, interpolation=transforms.InterpolationMode.NEAREST
                )
        
        img = self.normalize(img)
        
        return img.float(), mask.float()

class KvasirDataModule(LightningDataModule):
    def __init__(self, img_paths, msk_paths, batch_size=16, num_workers=2, 
                 train_split=0.7, val_split=0.15, img_size=(384, 384)):
        super().__init__()
        self.img_paths = img_paths
        self.msk_paths = msk_paths
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.train_split = train_split
        self.val_split = val_split
        self.img_size = img_size
        
        self.train_transform = SegmentationTransform(size=img_size, augment=True)
        self.val_test_transform = SegmentationTransform(size=img_size, augment=False)
        
        self.train_dataset = None
        self.val_dataset = None
        self.test_dataset = None
    
    def setup(self, stage=None):
        n_samples = len(self.img_paths)
        indices = list(range(n_samples))
        
        np.random.shuffle(indices)

        train_end = int(self.train_split * n_samples)
        val_end = train_end + int(self.val_split * n_samples)

        train_indices = indices[:train_end]
        val_indices = indices[train_end:val_end]
        test_indices = indices[val_end:]

        train_imgs = [self.img_paths[i] for i in train_indices]
        train_msks = [self.msk_paths[i] for i in train_indices]
        
        val_imgs = [self.img_paths[i] for i in val_indices]
        val_msks = [self.msk_paths[i] for i in val_indices]
        
        test_imgs = [self.img_paths[i] for i in test_indices]
        test_msks = [self.msk_paths[i] for i in test_indices]
        
        self.train_dataset = KvasirDataset(train_imgs, train_msks, self.train_transform)
        self.val_dataset = KvasirDataset(val_imgs, val_msks, self.val_test_transform)
        self.test_dataset = KvasirDataset(test_imgs, test_msks, self.val_test_transform)
        
        print(f"Rozmiary zbiorów: Train: {len(self.train_dataset)}, "
              f"Val: {len(self.val_dataset)}, Test: {len(self.test_dataset)}")
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=torch.cuda.is_available()
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=torch.cuda.is_available()
        )
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=torch.cuda.is_available()
        )


dm = KvasirDataModule(
    img_paths=imgs,
    msk_paths=msks,
    batch_size=16,
    num_workers=2,
    train_split=0.7,
    val_split=0.15,
    img_size=(384, 384)
)

dm.setup()

train_loader = dm.train_dataloader()
batch_imgs, batch_masks = next(iter(train_loader))

print(f"Kształt obrazów: {batch_imgs.shape}")
print(f"Kształt masek: {batch_masks.shape}")
print(f"Zakres wartości obrazów: [{batch_imgs.min():.3f}, {batch_imgs.max():.3f}]")
print(f"Unikalne wartości w maskach: {torch.unique(batch_masks)}")

Rozmiary zbiorów: Train: 700, Val: 150, Test: 150
Kształt obrazów: torch.Size([16, 3, 384, 384])
Kształt masek: torch.Size([16, 1, 384, 384])
Zakres wartości obrazów: [-1.000, 1.000]
Unikalne wartości w maskach: tensor([0., 1.])


## Zadanie 2 – Implementacja U-Net: architektura enkodera i dekodera

W tym zadaniu zaimplementujesz architekturę **U-Net** – jedną z najpopularniejszych sieci do segmentacji medycznej. U-Net składa się z:
- **Enkodera** (downsampling path): ekstraktuje cechy wysokiego poziomu, zmniejszając rozdzielczość.
- **Dekodera** (upsampling path): odbudowuje rozdzielczość, tworząc maskę segmentacji.
- **Skip connections**: połączenia z enkodera do dekodera na tym samym poziomie rozdzielczości – pozwalają zachować szczegóły przestrzenne.

![U-Net architecture diagram](https://iq.opengenus.org/content/images/2021/12/1_ovEGmOI3bcCeauu8jEBzsg.png)

**Blok DoubleConv:**
1. Zaimplementuj klasę `DoubleConv`, która dziedziczy po `nn.Module`.
2. Przyjmuje parametry: `in_ch` (kanały wejściowe), `out_ch` (kanały wyjściowe), `dropout` (prawdopodobieństwo dropout, domyślnie 0.0).
Dropout to technika regularyzacji, która losowo "wyłącza" pewien procent neuronów podczas treningu. Zmusza to sieć do uczenia się bardziej uniwersalnych i odpornych reprezentacji, ponieważ nie może polegać na pojedynczych neuronach/kanałach. Będziemy wykorzystywać `Dropout2d`, który usuwa całe kanały w warstwach konwolucyjnych. Warstwy te pozwalają ograniczyć zjawisko przeuczenia (overfitting).
3. Zbuduj sekwencję warstw:
   - `Conv2d(in_ch, out_ch, kernel_size=3, padding=1)` – konwolucja bez zmiany rozmiaru.
   - `BatchNorm2d(out_ch)` – normalizacja batch.
   - `ReLU(inplace=True)` – aktywacja.
   - Opcjonalnie: `Dropout2d(p=dropout)` jeśli dropout > 0.
   - Powtórz: `Conv2d(out_ch, out_ch, 3, padding=1)`, `BatchNorm2d`, `ReLU`. Ewentualnie może tutaj również nastąpić zmiana liczby kanałów wyjściowych. Wtedy na wejściu podczas inicjalizacji musimy przekazać dodatkowy parametr.
   - Opcjonalnie: ponownie `Dropout2d(p=dropout)` jeśli dropout > 0.
4. W metodzie `forward(x)` przepuść dane przez zbudowaną sekwencję.

**Model UNetSmall:**
1. Zaimplementuj klasę `UNetSmall`, która dziedziczy po `nn.Module`.
2. W `__init__` przyjmij: `in_ch1=3` (RGB), `out_ch=1` (maska binarna), `dropout=0.0`.
3. **Encoder (ścieżka downsampling):**
   - `DoubleConv(in_ch1, out_ch1, dropout)` – pierwszy blok (d1).
   - `nn.MaxPool2d(2)` – pooling 2×2 (zmniejsza rozdzielczość o połowę) (p1).
   - `DoubleConv(out_ch1, out_ch2, dropout)` – drugi blok (d2).
   - `nn.MaxPool2d(2)` (p2).
   - `DoubleConv(out_ch2, out_ch3, dropout)` – trzeci blok (d3).
   - `nn.MaxPool2d(2)` (p3).
4. **Bottleneck (najgłębsza część):**
   - `DoubleConv(out_ch3, out_ch4, dropout)` – ewentualnie w tej warstwie można zwiększyć dropout względem innych warstw (b).
5. **Decoder (ścieżka upsampling):**
   - `nn.ConvTranspose2d(out_ch4, out_ch3, kernel_size=2, stride=2)` – upsampling (zwiększa rozdzielczość 2×) (u3).
   - `DoubleConv(2*out_ch3, out_ch3, dropout)` – uwaga: 2*out_ch3 = out_ch3 (z poprzedniej warstwy) + out_ch3 (skip connection z enkodera) (c3).
   - `nn.ConvTranspose2d(out_ch3, out_ch2, 2, 2)` (u2).
   - `DoubleConv(2*out_ch2, out_ch2, dropout)` (c2).
   - `nn.ConvTranspose2d(out_ch2, out_ch1, 2, 2)` (u1).
   - `DoubleConv(2*out_ch1, out_ch1, dropout)` (c1).
6. **Warstwa wyjściowa:**
   - `nn.Conv2d(out_ch1, out_ch, kernel_size=1)` – konwolucja 1×1, zwraca logity (out).
7. W metodzie `forward(x)`:
   - **Encoder:** przepuść wejście przez `d1→p1→d2→p2→d3→p3→b`, zapamiętuj wyjścia z d1, d2 i d3, bo są potrzebne do skip connections.
   - **Decoder:**
     - Przepuść wyjście z bottleneck przez u3, potem skonkatenuj ze skip connection z d3 `torch.cat([u3, d3], dim=1)`, a następnie przepuść przez c3.
     - Zrób do samo wykorzystując warstwy u2 i c2 oraz wyjście z d2.
     - Zrób do samo wykorzystując warstwy u1 i c1 oraz wyjście z d1.
   - Na koniec przepuść dane przez warstwę wyjściową out.

**Dice Loss:**
1. Zaimplementuj klasę `DiceLoss`, dziedziczącą po `nn.Module`.
2. W `__init__(self, eps=1e-6)` zapamiętaj `eps=1e-6` (wartość dodawana dla stabilności numerycznej).
3. DiceLoss pochodzi od współczynnika Dice (Dice coefficient, F1 dla segmentacji), który mierzy nakładanie się dwóch masek: `Dice = 2 * |P ∩ G| / (|P| + |G|)`, gdzie `P` to predykcja, `G` to maska, natomiast operator | | oznacza liczność zbioru. Zakres wartości to [0, 1], im większe — tym lepsze dopasowanie. `DiceLoss` wynosi `1 − Dice` (dla zadanie minimalizacji). Dodajemy małą wartość eps dla stabilności numerycznej do licznika i mianownika. Skupia się on na nakładaniu się obszarów, a nie na pojedynczych pikselach, co daje lepszą jakość predykcji maski.
4. W `forward(self, logits, targets)`:
   - Zastosuj sigmoid do logitów: `torch.sigmoid(logits)`.
   - Upewnij się że targets są typu float.
   - Oblicz licznik. W tym celu pomnóż wyniki sigmoidy z `targets`, a następnie zsumuj po wymiarach przestrzennych, pomnóż przez 2 i dodał `eps`.
   - Oblicz mianownik symetrycznie do licznika, zgodnie z wzorem.
   - Podziel otrzymane wartości, oblicz wartość średnią po wszystkich wymiarach (batch) i odejmij wynik od 1. Zwróć obliczoną wielkość.

**Callbacks i Lightning Module:**
1. Kod zawiera już gotową implementację `MetricsCallback` (do zbierania metryk).
2. Konieczna jest jeszcze implementacja `LitUNet` (Lightning wrapper dla zaprejektowanej sieci z optymalizatorem, stratami i metrykami), która dziedziczy po `LightningModule`. Implementacja jest podobna do poprzedniego ćwiczenia.
3. W metodzie `__init__(self, in_ch=3, lr=1e-3, dropout=0.0)`:
   - Wywołaj `super().__init__()`.
   - Zapisz hiperparametry: `self.save_hyperparameters()` – Lightning automatycznie zapisze je w checkpointach.
   - Stwórz instancję stworzonej wcześniej sieci.
   - Zdefiniuj i zapamiętaj funkcje straty:
     - `nn.BCEWithLogitsLoss()` – Binary Cross-Entropy (działa na logitach, wewnętrznie aplikuje sigmoid).
     - `DiceLoss()` – nasza implementacja Dice Loss.
   - Zdefiniuj metryki (z torchmetrics):
     - `BinaryJaccardIndex()` – IoU (Intersection over Union).
     - `BinaryF1Score()` – F1/Dice Score. Odpowiednik zaimplementowanej fukncji straty, ale używa progowania prawdopodobieństwa, przez co nie można dla niej obliczyć gradientu.
4. W metodzie `forward(self, x)`:
   - Przepuść wejście przez sieć.
5. W metodzie `configure_optimizers(self)`:
   - Wykorzystaj optymalizator `Adam`.
   - Wykorzystaj scheduler `CosineAnnealingLR`.
   - Zwróć słownik zawierający optymalizator i scheduler.
6. Zdefiniuj metody `training_step`, `validation_step` i `test_step`. Prawie wszystkie operacje są w nich takie sameg więc można je przenieść do funkcji pomocniczej.
   - Rozpakuj batch (obrazy i maski).
   - Oblicz wyjście sieci `self(x)`.
   - Oblicz stratę jako sumę BCEWithLogitsLoss + DiceLoss
   - Oblicz metryki do monitoringu w bloku `with torch.no_grad():`:
     - Wykonaj progowanie predykcji sieci, aby wyznaczyć wynik segmentacji.
     - Upewnij się że zarówno maska groundtruth i wynik predykcji są typu `int`.
     - Oblicz IoU i F1(Dice) za pomocą stworzonych podczas inicjalizacji obiektów.
   - Zaloguj metryki za pomocą funkcji `self.log()`:
     - Pierwszym argumentem jest nazwa logowanej metryki.
     - Drugim argumentem jest jej wartość.
     - Flaga `on_epoch` sprawia, że metryka jest agregowana i logowana dla całej epoki. Ustaw ją jako `True`.
     - Flaga `prog_bar` mówi czy wartość tej metryki ma być wyświetlona obok paska postępu.
   - Zwróć `loss` dla `training_step`.

In [4]:
class MetricsCallback(pl.Callback):
    """Callback to collect training metrics for visualization"""
    def __init__(self):
        super().__init__()
        self.metrics = {
            'train_loss': [],
            'val_loss': [],
            'train_dice': [],
            'val_dice': [],
            'train_iou': [],
            'val_iou': [],
            'epoch': []
        }
    
    def on_train_epoch_end(self, trainer, pl_module):
        # Collect training metrics
        metrics = trainer.callback_metrics
        epoch = trainer.current_epoch + 1
        
        self.metrics['epoch'].append(epoch)
        self.metrics['train_loss'].append(metrics.get('train_loss_epoch', float('nan')).item())
        self.metrics['train_dice'].append(metrics.get('train_dice_epoch', float('nan')).item())
        self.metrics['train_iou'].append(metrics.get('train_iou_epoch', float('nan')).item())
    
    def on_validation_epoch_end(self, trainer, pl_module):
        # Collect validation metrics
        metrics = trainer.callback_metrics
        
        # Only add validation metrics if we have them
        if 'val_loss' in metrics:
            self.metrics['val_loss'].append(metrics['val_loss'].item())
            self.metrics['val_dice'].append(metrics['val_dice'].item())
            self.metrics['val_iou'].append(metrics['val_iou'].item())


In [5]:
# Blok DoubleConv
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch, dropout=0.0):
        super().__init__()
        layers = [
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        ]
        
        if dropout > 0:
            layers.append(nn.Dropout2d(p=dropout))
            
        layers.extend([
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        ])
        
        if dropout > 0:
            layers.append(nn.Dropout2d(p=dropout))
            
        self.conv = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.conv(x)

# Model UNetSmall
class UNetSmall(nn.Module):
    def __init__(self, in_ch=3, out_ch=1, dropout=0.0):
        super().__init__()
        out_ch1 = 64
        out_ch2 = 128
        out_ch3 = 256
        out_ch4 = 512
        
        # Encoder
        self.d1 = DoubleConv(in_ch, out_ch1, dropout)
        self.p1 = nn.MaxPool2d(2)
        
        self.d2 = DoubleConv(out_ch1, out_ch2, dropout)
        self.p2 = nn.MaxPool2d(2)
        
        self.d3 = DoubleConv(out_ch2, out_ch3, dropout)
        self.p3 = nn.MaxPool2d(2)
        
        # Bottleneck
        self.b = DoubleConv(out_ch3, out_ch4, dropout)
        
        # Decoder
        self.u3 = nn.ConvTranspose2d(out_ch4, out_ch3, kernel_size=2, stride=2)
        self.c3 = DoubleConv(2 * out_ch3, out_ch3, dropout)
        
        self.u2 = nn.ConvTranspose2d(out_ch3, out_ch2, kernel_size=2, stride=2)
        self.c2 = DoubleConv(2 * out_ch2, out_ch2, dropout)
        
        self.u1 = nn.ConvTranspose2d(out_ch2, out_ch1, kernel_size=2, stride=2)
        self.c1 = DoubleConv(2 * out_ch1, out_ch1, dropout)
        
        # Output
        self.out = nn.Conv2d(out_ch1, out_ch, kernel_size=1)
    
    def forward(self, x):
        # Encoder
        d1_out = self.d1(x)
        p1_out = self.p1(d1_out)
        
        d2_out = self.d2(p1_out)
        p2_out = self.p2(d2_out)
        
        d3_out = self.d3(p2_out)
        p3_out = self.p3(d3_out)
        
        # Bottleneck
        b_out = self.b(p3_out)
        
        # Decoder
        u3_out = self.u3(b_out)
        c3_in = torch.cat([u3_out, d3_out], dim=1)
        c3_out = self.c3(c3_in)
        
        u2_out = self.u2(c3_out)
        c2_in = torch.cat([u2_out, d2_out], dim=1)
        c2_out = self.c2(c2_in)
        
        u1_out = self.u1(c2_out)
        c1_in = torch.cat([u1_out, d1_out], dim=1)
        c1_out = self.c1(c1_in)
        
        # Output
        out = self.out(c1_out)
        
        return out

# Dice Loss
class DiceLoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.eps = eps
    
    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        targets = targets.float()
        probs_flat = probs.view(probs.size(0), -1)
        targets_flat = targets.view(targets.size(0), -1)
        intersection = 2 * (probs_flat * targets_flat).sum(dim=1)
        union = probs_flat.sum(dim=1) + targets_flat.sum(dim=1)
        dice = (intersection + self.eps) / (union + self.eps)
        
        return 1 - dice.mean()

# Lightning Module dla U-Net
# Lightning Module dla U-Net - POPRAWIONA WERSJA
class LitUNet(LightningModule):
    def __init__(self, in_ch=3, lr=1e-3, dropout=0.0):
        super().__init__()
        self.save_hyperparameters()
        
        self.model = UNetSmall(in_ch=in_ch, out_ch=1, dropout=dropout)
        
        # Funkcje straty
        self.bce_loss = nn.BCEWithLogitsLoss()
        self.dice_loss = DiceLoss()
        
        # Metryki - używamy _epoch do agregacji
        self.train_iou = BinaryJaccardIndex()
        self.val_iou = BinaryJaccardIndex()
        self.test_iou = BinaryJaccardIndex()
        
        self.train_f1 = BinaryF1Score()
        self.val_f1 = BinaryF1Score()
        self.test_f1 = BinaryF1Score()
        
        self.lr = lr
    
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=10, eta_min=1e-6
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler
        }
    
    def _shared_step(self, batch, batch_idx, prefix):
        x, y_true = batch
        
        # Predykcja
        y_pred = self(x)
        
        # Obliczanie strat
        bce_loss = self.bce_loss(y_pred, y_true)
        dice_loss = self.dice_loss(y_pred, y_true)
        total_loss = bce_loss + dice_loss
        
        # Obliczanie metryk
        with torch.no_grad():
            y_pred_bin = (torch.sigmoid(y_pred) > 0.5).float()
            y_true_int = y_true.int()
            y_pred_int = y_pred_bin.int()
            
            if prefix == 'train':
                iou = self.train_iou(y_pred_int, y_true_int)
                f1 = self.train_f1(y_pred_int, y_true_int)
            elif prefix == 'val':
                iou = self.val_iou(y_pred_int, y_true_int)
                f1 = self.val_f1(y_pred_int, y_true_int)
            else:  # test
                iou = self.test_iou(y_pred_int, y_true_int)
                f1 = self.test_f1(y_pred_int, y_true_int)
        
        # Logowanie - POPRAWIONE dla zgodności z MetricsCallback
        self.log(f'{prefix}_loss', total_loss, on_epoch=True, prog_bar=True)
        self.log(f'{prefix}_loss_epoch', total_loss, on_epoch=True, prog_bar=False)
        self.log(f'{prefix}_bce_loss', bce_loss, on_epoch=True)
        self.log(f'{prefix}_dice_loss', dice_loss, on_epoch=True)
        
        # Logowanie F1 jako dice dla zgodności z MetricsCallback
        self.log(f'{prefix}_dice', f1, on_epoch=True, prog_bar=True)
        self.log(f'{prefix}_dice_epoch', f1, on_epoch=True, prog_bar=False)
        
        # Logowanie IoU
        self.log(f'{prefix}_iou', iou, on_epoch=True, prog_bar=True)
        self.log(f'{prefix}_iou_epoch', iou, on_epoch=True, prog_bar=False)
        
        return total_loss
    
    def training_step(self, batch, batch_idx):
        return self._shared_step(batch, batch_idx, 'train')
    
    def validation_step(self, batch, batch_idx):
        return self._shared_step(batch, batch_idx, 'val')
    
    def test_step(self, batch, batch_idx):
        return self._shared_step(batch, batch_idx, 'test')


    
# test
batch_size = 2
x_test = torch.randn(batch_size, 3, 384, 384)
y_test = torch.rand(batch_size, 1, 384, 384)
y_test = (y_test > 0.5).float()

print(f"Unikalne wartości w y_test: {torch.unique(y_test)}")

# Test modelu
model = UNetSmall(in_ch=3, out_ch=1, dropout=0.1)
y_pred = model(x_test)
print(f"Input shape: {x_test.shape}")
print(f"Output shape: {y_pred.shape}")

# Test funkcji straty
dice_loss = DiceLoss()
loss_value = dice_loss(y_pred, y_test)
print(f"Dice loss: {loss_value:.4f}")

# Test Lightning Module
lit_model = LitUNet(in_ch=3, lr=1e-3, dropout=0.1)

# Test optymalizatora
optimizer = lit_model.configure_optimizers()
print("Optymalizator i scheduler skonfigurowane poprawnie")

# Test pojedynczego kroku treningowego
with torch.no_grad():
    loss = lit_model.training_step((x_test, y_test), 0)
    print(f"Training step loss: {loss:.4f}")

# Test MetricsCallback
metrics_callback = MetricsCallback()
print("MetricsCallback zainicjalizowany poprawnie")

print("Wszystkie testy zakończone pomyślnie!")

Unikalne wartości w y_test: tensor([0., 1.])
Input shape: torch.Size([2, 3, 384, 384])
Output shape: torch.Size([2, 1, 384, 384])
Dice loss: 0.5064
Optymalizator i scheduler skonfigurowane poprawnie
Training step loss: 1.1849
MetricsCallback zainicjalizowany poprawnie
Wszystkie testy zakończone pomyślnie!


/home/karolina/studia/piaom/venv/lib/python3.10/site-packages/pytorch_lightning/core/module.py:449: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


## Zadanie 3 – Trening U-Net: callbacks, wizualizacja metryk i ewaluacja

Teraz chcemy wytrenować stworzoną architekturę sieci do segmentacji.

1. Stworzenie modelu i callbacków:
   - Utwórz instancję klasy sieci.
   - Utwórz obiekty `MetricsCallback` – będzie zbierać metryki do wizualizacji.
   - Utwórz `EarlyStopping` – ma zatrzymać trening jeśli metryka F1/Dice na zbiorze walidacyjnym nie rośnie przez określoną liczbę epok.
   - Utwórz `ModelCheckpoint` – ma zapisywać najlepszy model pod względem metryki F1/Dice dla zbioru walidacyjnego.

2. **Trening:**
   - Stwórz `Trainer(max_epochs=50, accelerator='auto', devices=1, callbacks=[metrics_callback, early, ckpt], precision=16)`. Określ maksymalną liczbę epok, ustaw `accelerator` jako `'auto'`, przekaż callbacki w `callbacks`. Możesz również sprawdzić jak precyzja numeryczna wpływa na szybkość i wyniki uczenia. Jest ona przekazywana przez argument `precision`. Porównaj wartość 32 i 16 (stosowanie Automatic Mixed Precision).
   - Wykonaj trening sieci `trainer.fit`.

3. **Wizualizacja przebiegu treningu:**
   - Wyciągnij zebrane metryki z `metrics_callback.metrics`. Jest to słownik zawierający nazwy pól jak w wywołaniach metod `self.log` sieci.
   - Stwórz 3 wykresy:
     -  Train Loss i Val Loss w funkcji epok.
     -  Train Dice i Val F1/Dice w funkcji epok.
     -  Train IoU i Val IoU w funkcji epok.

4. **Test na zbiorze testowym:**
   - Wczytaj najlepszy model.
   - Sprawdź metryki na zbiorze testowym za pomocą metody `test`.

5. **Wizualizacja predykcji:**
   - Przełącz model w tryb ewaluacji.
   - Pobierz jeden batch z test loadera: `xb, yb = next(iter(dm.test_dataloader()))`.
   - Oblicz predykcje dla pobranego batcha wewnątrz bloku `torch.no_grad():`. Dodatkowo obliczone logity przepuść przez sigmoidę, aby otrzymać wartości prawdopodobieństwa.
   - Zbinaryzuj predykcje.
   - Dla kilku pierwszych przykładów wyświetl w rzędzie:
     - Kolumna 1: Oryginalny obraz: `((xb[i].permute(1,2,0).numpy()*0.5)+0.5).clip(0,1)`.
     - Kolumna 2: Maska GT: `yb[i,0]`.
     - Kolumna 3: Predykcja: `preds[i,0]`.
     - Kolumna 4: Obraz oryginalny z nałożoną predykcją (zmień kolor pikseli na czerwony):
     `overlay = img.copy(); m = preds[i,0].numpy()>0.5; overlay[m] = [1.0, 0.0, 0.0]`

In [6]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [ ]:
# Zadanie 3 – Trening U-Net: callbacks, wizualizacja metryk i ewaluacja

print("=== U-NET TRAINING ===")

print("\n1. Initializing model and callacks...")

dm = KvasirDataModule(
    img_paths=imgs,
    msk_paths=msks,
    batch_size=4,
    num_workers=2,
    train_split=0.7,
    val_split=0.15,
    img_size=(384, 384)
)

model = LitUNet(in_ch=3, lr=1e-3, dropout=0.2)

metrics_callback = MetricsCallback()

early_stopping = EarlyStopping(
    monitor='val_dice',
    mode='max',
    patience=10,
    verbose=True
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_dice',  # Changed from 'val_f1' to 'val_dice'
    mode='max',
    save_top_k=1,
    save_last=True,
    filename='unet-best-{epoch:02d}-{val_dice:.3f}',  # Also updated filename
    verbose=True
)

print("Created model and callbacks.")

trainer = Trainer(
    max_epochs=50,
    accelerator='auto',
    devices=1,
    callbacks=[metrics_callback, early_stopping, checkpoint_callback],
    precision=16,
    log_every_n_steps=10,
    val_check_interval=0.5
)

print("Training...")
trainer.fit(model, dm)

print("Training finished.")

# 3. Wizualizacja przebiegu treningu
print("\n3. Metrics visualization")

metrics = metrics_callback.metrics

if metrics['epoch']:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    axes[0].plot(metrics['epoch'], metrics['train_loss'], 'b-', label='Train Loss', linewidth=2)
    axes[0].plot(metrics['epoch'][:len(metrics['val_loss'])], metrics['val_loss'], 'r-', label='Val Loss', linewidth=2)
    axes[0].set_xlabel('Epoka')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Train and Val Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(metrics['epoch'], metrics['train_dice'], 'b-', label='Train Dice', linewidth=2)
    axes[1].plot(metrics['epoch'][:len(metrics['val_dice'])], metrics['val_dice'], 'r-', label='Val Dice', linewidth=2)
    axes[1].set_xlabel('Epoka')
    axes[1].set_ylabel('Dice/F1 Score')
    axes[1].set_title('Train and Val Dice Score')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    axes[2].plot(metrics['epoch'], metrics['train_iou'], 'b-', label='Train IoU', linewidth=2)
    axes[2].plot(metrics['epoch'][:len(metrics['val_iou'])], metrics['val_iou'], 'r-', label='Val IoU', linewidth=2)
    axes[2].set_xlabel('Epoka')
    axes[2].set_ylabel('IoU')
    axes[2].set_title('Train and Val IoU')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    best_val_f1 = max(metrics['val_dice']) if metrics['val_dice'] else 0
    best_val_iou = max(metrics['val_iou']) if metrics['val_iou'] else 0
    best_epoch = metrics['epoch'][metrics['val_dice'].index(best_val_f1)] if metrics['val_dice'] else 0
    
    print(f"\nBest val results:")
    print(f"Epoka: {best_epoch}")
    print(f"Val Dice/F1: {best_val_f1:.4f}")
    print(f"Val IoU: {best_val_iou:.4f}")
else:
    print("No data to visualize...")


print("\n4. Testing...")

if checkpoint_callback.best_model_path:
    print(f"Loading the best model: {checkpoint_callback.best_model_path}")
    best_model = LitUNet.load_from_checkpoint(checkpoint_callback.best_model_path)
else:
    print("Using latest model...")
    best_model = model

test_results = trainer.test(best_model, dm)
print("Testing done")


print("\n5. Visualizing predictions...")

best_model.eval()

test_loader = dm.test_dataloader()
xb, yb = next(iter(test_loader))

with torch.no_grad():
    logits = best_model(xb)
    probs = torch.sigmoid(logits)
    preds = (probs > 0.5).float()

print(f"Visualizing batch: {xb.shape}")

n_examples = min(4, xb.shape[0])

fig, axes = plt.subplots(n_examples, 4, figsize=(16, 4 * n_examples))

if n_examples == 1:
    axes = axes.reshape(1, -1)

for i in range(n_examples):
    img = xb[i].permute(1, 2, 0).numpy()
    img = (img * 0.5) + 0.5
    img = np.clip(img, 0, 1)
    
    gt_mask = yb[i, 0].numpy()
    
    pred_mask = preds[i, 0].numpy()
    
    overlay = img.copy()
    m = pred_mask > 0.5
    overlay[m] = [1.0, 0.0, 0.0]
    
    axes[i, 0].imshow(img)
    axes[i, 0].set_title(f'Obraz {i+1}')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(gt_mask, cmap='gray')
    axes[i, 1].set_title(f'GT Maska {i+1}')
    axes[i, 1].axis('off')
    
    axes[i, 2].imshow(pred_mask, cmap='gray')
    axes[i, 2].set_title(f'Predykcja {i+1}')
    axes[i, 2].axis('off')
    
    axes[i, 3].imshow(overlay)
    axes[i, 3].set_title(f'Overlay {i+1}')
    axes[i, 3].axis('off')

plt.suptitle('Visualizing predictions on test dataset', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print("\n6. Analyzing prediction quality...")

with torch.no_grad():
    test_iou = BinaryJaccardIndex()(preds.int(), yb.int())
    test_f1 = BinaryF1Score()(preds.int(), yb.int())
    
    print(f"Prediction quality on a test batch:")
    print(f"IoU: {test_iou:.4f}")
    print(f"F1/Dice: {test_f1:.4f}")

print("\n7. Checkpoints:")
if checkpoint_callback.best_model_path:
    print(f"Best model: {checkpoint_callback.best_model_path}")
if hasattr(checkpoint_callback, 'last_model_path') and checkpoint_callback.last_model_path:
    print(f"Latest model: {checkpoint_callback.last_model_path}")

print("\n" + "="*50)
print("FINISHED TRAINING!")
print("="*50)

/home/karolina/studia/piaom/venv/lib/python3.10/site-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/karolina/studia/piaom/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') th

=== U-NET TRAINING ===

1. Initializing model and callacks...
Created model and callbacks.
Training...
Rozmiary zbiorów: Train: 700, Val: 150, Test: 150
Epoch 0:  50%|████▉     | 87/175 [00:28<00:28,  3.05it/s, v_num=9, train_loss_step=1.090, train_dice_step=0.237, train_iou_step=0.134]  

Metric val_dice improved. New best score: 0.304


Epoch 0:  50%|████▉     | 87/175 [00:32<00:32,  2.68it/s, v_num=9, train_loss_step=1.090, train_dice_step=0.237, train_iou_step=0.134, val_loss=1.120, val_dice=0.304, val_iou=0.186]

Epoch 0, global step 87: 'val_dice' reached 0.30409 (best 0.30409), saving model to '/home/karolina/studia/piaom/lab5/lightning_logs/version_9/checkpoints/unet-best-epoch=00-val_dice=0.304.ckpt' as top 1


...

Epoch 23:  99%|█████████▉| 174/175 [00:31<00:00,  5.47it/s, v_num=9, train_loss_step=0.672, train_dice_step=0.753, train_iou_step=0.604, val_loss=0.976, val_dice=0.484, val_iou=0.328, train_loss_epoch=0.986, train_dice_epoch=0.482, train_iou_epoch=0.327]

Monitored metric val_dice did not improve in the last 10 records. Best score: 0.499. Signaling Trainer to stop.


Epoch 23:  99%|█████████▉| 174/175 [00:34<00:00,  5.11it/s, v_num=9, train_loss_step=0.672, train_dice_step=0.753, train_iou_step=0.604, val_loss=0.987, val_dice=0.465, val_iou=0.314, train_loss_epoch=0.986, train_dice_epoch=0.482, train_iou_epoch=0.327]

Epoch 23, global step 4199: 'val_dice' was not in top 1


Epoch 23:  99%|█████████▉| 174/175 [00:34<00:00,  5.09it/s, v_num=9, train_loss_step=0.672, train_dice_step=0.753, train_iou_step=0.604, val_loss=0.987, val_dice=0.465, val_iou=0.314, train_loss_epoch=0.979, train_dice_epoch=0.491, train_iou_epoch=0.336]


## Zadanie 4 – Transfer Learning: U-Net z pre-trained ResNet Encoder

Znowu wykorzystamy transfer learning do zbudowania sieci o większej skuteczności. Zamiast używać całej nauczonej wcześniej sieci wyciągniemy tylko kilka warstw i użyjemy ich w enkoderze modelu U-Net.

1. **ResNetEncoder:**
   - Zaimplementuj klasę `ResNetEncoder`, która dziedziczy po `nn.Module`.
   - W `__init__` przyjmij: `pretrained=True` (czy ładować pretrenowane wagi), `freeze_encoder=False` (czy wczytane wagi mają być zamrożone).
   - Wczytaj pretrained ResNet18: `models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)`.
   - Wyciągnij warstwy enkodera:
     - `resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool` – początkowe warstwy.
     - `resnet.layer1, resnet.layer2, resnet.layer3, resnet.layer4` – bloki ResNet (64, 128, 256, 512 kanałów).
   - Jeśli `freeze_encoder=True`, ustaw wszystkie parametry na `requires_grad=False` wewnątrz bloku `for param in self.parameters():`
   - W metodzie `forward(x)` przepuść dane przez warstwy i zwróć 5 tensorów skip connections: `x1` (po conv1), `x2` (po layer1), `x3` (po layer2), `x4` (po layer3), `x5` (po layer4/bottleneck).

2. **UNetResNet:**
   - Zaimplementuj klasę `UNetResNet`, która dziedziczy po `nn.Module`.
   - W `__init__` przyjmij: `pretrained=True`, `freeze_encoder=False`, `dropout`.
   - Stwórz encoder: `self.encoder = ResNetEncoder(pretrained, freeze_encoder)`.
   - Zbuduj decoder (podobnie jak w poprzednim zadaniu):
     - `ConvTranspose2d` upsampling z bottleneck (u1).
     - `DoubleConv` pamiętaj o konkatenacji wyjścia z u1 i skip connection z x4.
     - `ConvTranspose2d` (u2).
     - `DoubleConv` u2 + x3.
     - `ConvTranspose2d` (u3).
     - `DoubleConv` u3 + x2.
     - `ConvTranspose2d` (u4).
     - `DoubleConv` u4 + x1.
     - `ConvTranspose2d` powrót do orygionalej rozdzielczości.
     - `DoubleConv`.
   - Warstwa wyjściowa `Conv2d` z jednym kanałem wyjściowym o rozmiarze 1x1.
   - W metodzie `forward(x)`:
     - Wywołaj encoder: `x1, x2, x3, x4, x5 = self.encoder(x)`.
     - Przepuść przez decoder i skip connections.
     - Zwróć logity.

3. **LitUNetResNet:**
   - Zaimplementuj Lightning wrapper analogicznie do `LitUNet`.
   - Dodaj metody pomocnicze:
     - `freeze_encoder()`: zamraża wagi enkodera (tylko decoder będzie trenowany).
     - `unfreeze_encoder()`: odmraża encoder (cała sieć będzie trenowana).

4. **Test architektury:**
   - Stwórz instancję `UNetResNet`.
   - Przepuść losowy tensor przez sieć i sprawdź kształt wyjścia.
   - Wyświetl osobno liczbę wszystkich oraz liczbę uczonych parametrów `sum(p.numel() for p in model_tl.parameters() if p.requires_grad)`

## Zadanie 5 – Dwuetapowy Transfer Learning: Frozen Encoder

Trening chcemy wykonać podobnie jak w poprzednim ćwiczeniu, czyli zaczynamy od zamrożonych wag z transfer learningu, a potem wykonujemy fine-tuning. Proces treningu jest taki sam jak dla naszej wcześniejszej sieci.

**Trening z zamrożonym encoderem:**
1. Stwórz obiekt klasy `LitUNetResNet` z pretrenowanymi i zamrożonymi wagami.
2. Stwórz callbacki: `MetricsCallback`, `EarlyStopping`, `ModelCheckpoint`.
3. Stwórz `Trainer`.
4. Wywołaj trening sieci wywołując metodę `fit`
5. Wyświetl najlepszy checkpoint i val_dice.
6. Sprawdź oraz wyświetl metryki (3 wykresy: loss, dice, iou) podobnie jak we wcześniejszym treningu.
7. Wczytaj najlepszy model sieci i wykonaj dla niego testy na zbiorze testowym.
8. Wyświetl przykładowe predykcje (4 obrazy × 4 kolumny).

## Zadanie 6 – Dwuetapowy Transfer Learning: Fine-tuning

**Fine-tuning całej sieci:**
1. Odmroź encoder dla najlepszego modelu z poporzedniej części.
2. Ustaw learning rate: `best_stage1.hparams.lr = ` (podczas fine-tuningu zazwyczaj jest mniejszy).
3. Zaktualizuj hparam: `best_stage1.hparams.freeze_encoder = False`.
4. Stwórz nowe callbacki: `MetricsCallback`, `EarlyStopping`, `ModelCheckpoint`.
5. Stwórz nowy `Trainer`.
6. Wykonaj trening wywołując metodę `fit`.
7. Sprawdź oraz wyświetl metryki (3 wykresy: loss, dice, iou) podobnie jak we wcześniejszym treningu.
8. Wczytaj najlepszy model sieci i wykonaj dla niego testy na zbiorze testowym.
9. Wyświetl przykładowe predykcje (4 obrazy × 4 kolumny).

---

## Podsumowanie i wnioski

Porównaj wyniki przeprowadzonych treningów sieci. Weź pod uwagę liczbę trenowanych parametrów.
Odpowiedz również krótko na poniższe pytania. 

1. Czym segmentacja różni się od zadania klasyfikacji?

2. Czym jest architektura U-Net? Czym ona się charakteryzuje?

3. Czym jest funkcja straty DiceLoss?